In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7488,2024-07-16,Colômbia Lbp,21:05,Titanes de Barranquilla,Motilones del Norte,1.49,2.50,157.5,1.82,1.88,0.0,0.00,0.00,pG0NzVUO,0.671141,0.400000,0.549451,0.531915,0.071141,103.476,17.225872,0.166472,3.0,0.000000,0.000000,126.84,1.312,0.134425,0.102458,43.0,167.428,52.903596,0.315978,0.6,1.341641,2.236068,169.05,2.314,0.890635,0.384890,-30.0,84,69,1.51,2.45,100.676,167.428,0.357984,0.022933,NaN,0.42,0.084,5.833333,0.804544,0.8,-0.004544,-3.57,-0.714,-2.100840,0.540141,0.4,-0.140141
7489,2024-07-16,Eua Nba,17:00,Phoenix Suns,Indiana Pacers,2.79,1.43,177.5,1.80,1.94,0.0,0.00,0.00,IqPHTxNt,0.358423,0.699301,0.555556,0.515464,0.057724,218.376,38.428939,0.175976,0.0,0.000000,NaN,270.10,2.342,0.805401,0.343894,-77.0,308.472,112.971292,0.366229,0.0,0.000000,NaN,223.72,2.966,0.914839,0.308442,-35.0,73,94,3.70,2.38,176.156,367.098,0.455766,0.052938,NaN,-3.34,-0.668,-2.679641,0.654878,0.5,-0.154878,-5.00,-1.000,-0.430000,0.497002,0.2,-0.297002
7490,2024-07-16,Eua Nba,17:30,Brooklyn Nets,New York Knicks,1.69,2.15,178.5,1.85,1.89,-3.5,1.99,2.52,bgXPRGig,0.591716,0.465116,0.540541,0.529101,0.056832,325.572,241.356490,0.741331,1.2,1.643168,1.369306,151.32,3.498,2.742010,0.783879,-28.0,280.650,57.891449,0.206276,0.6,1.341641,2.236068,306.90,2.788,0.749947,0.268991,-45.0,78,90,1.94,3.41,254.728,283.670,0.169411,0.015125,0.166194,1.55,0.310,2.225806,0.504236,0.6,0.095764,-0.13,-0.026,-44.230769,0.424760,0.5,0.075240
7491,2024-07-16,Eua Nba,19:00,Denver Nuggets,Charlotte Hornets,2.83,1.42,174.5,1.80,1.92,0.0,0.00,0.00,W6VXPfM5,0.353357,0.704225,0.555556,0.520833,0.057582,207.714,68.627152,0.330393,1.2,1.643168,1.369306,178.20,2.218,0.225544,0.101688,-23.0,498.818,262.682459,0.526610,1.8,1.643168,0.912871,123.14,4.540,2.106086,0.463896,-44.0,81,94,2.20,1.31,146.746,500.682,0.469186,0.045620,NaN,-0.85,-0.170,-10.764706,0.751554,0.6,-0.151554,8.62,1.724,0.243619,0.316866,0.4,0.083134
7492,2024-07-16,Eua Nba,19:30,Detroit Pistons,Chicago Bulls,2.16,1.68,173.5,1.82,1.92,1.5,1.99,1.52,SYOtOYjI,0.462963,0.595238,0.549451,0.520833,0.058201,359.754,178.306913,0.495636,1.2,1.643168,1.369306,254.04,3.674,1.437560,0.391279,-31.0,383.914,407.907628,1.062497,1.2,1.643168,1.369306,226.32,3.532,3.279218,0.928431,-10.0,87,82,2.92,2.76,454.838,382.198,0.176777,0.037813,0.189368,-5.00,-1.000,-1.160000,0.372623,0.2,-0.172623,-2.00,-0.400,-1.700000,0.450061,0.4,-0.050061
7493,2024-07-16,Eua Nba,21:00,Minnesota Timberwolves,Philadelphia 76ers,1.51,2.53,175.5,1.80,1.92,-4.5,1.91,2.71,zaSlMCLU,0.662252,0.395257,0.555556,0.520833,0.057509,168.706,45.501404,0.269708,1.2,1.643168,1.369306,164.85,1.662,0.342155,0.205869,-13.0,176.984,56.623902,0.319938,1.2,1.643168,1.369306,131.10,1.720,0.399562,0.232304,9.0,105,95,1.57,1.38,161.068,230.236,0.357054,0.045620,0.244885,-1.76,-0.352,-1.448864,0.616654,0.4,-0.216654,2.94,0.588,2.602041,0.479312,0.6,0.120688
7494,2024-07-16,Eua Nba,23:30,San Antonio Spurs,New Orleans Pelicans,1.99,1.79,168.5,1.82,1.90,1.5,1.87,1.67,OOWDMtXA,0.502513,0.558659,0.549451,0.526316,0.061172,361.012,294.951746,0.817014,2.4,1.341641,0.559017,165.11,3.768,3.310456,0.878571,0.0,214.664,35.935359,0.167403,0.6,1.341641,2.236068,263.16,2.468,0.464833,0.188344,-34.0,79,86,2.09,3.06,419.606,231.208,0.074826,0.030413,0.079899,4.67,0.934,1.059957,0.3728

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7490,17:30,Eua Nba,Brooklyn Nets,New York Knicks,1.69,Back Home
7494,23:30,Eua Nba,San Antonio Spurs,New Orleans Pelicans,1.99,Back Home
